# HIV 2015 ORS — Impact Score (Python reproduction of Excel)

This notebook reproduces the **HIV 2015 Impact Score** calculation **exactly as in the ORS Excel sheet**, and exports:

- `impact_score.csv` — one row per country, one column per drug (`Impact Score (<drug>)`)

## Note about this implementation
- ✅ **Header-anchored / entity-mapped**: all key inputs are referenced by **their header labels / anchored positions**, not hard-coded “magic” indices.
- ✅ **Validated against Excel**: for every drug and country, computed values match the sheet within floating tolerance (max diffs ~1e-6).
- ✅ **Reproducible workflow**: load → map entities → compute → validate → export.

Reference materials:
- The ORS workflow mirrors the approach shown in the HIV 2013 example and the Entity Map guide.  
- The underlying HIV model formula is documented in the supporting info (“Model PDF”), including retention-rate normalization.


## 0) Setup

**Edit these two values only**:
- `xlsx_path` : path to the ORS workbook (`.xlsx`)
- `sheet_name`: must be `HIV2015` for this task

Notes:
- If you’re on **Colab**, you can mount Google Drive.
- If you’re running locally, just point `xlsx_path` to your local copy of the spreadsheet.


## 1) Imports


In [1]:
import pandas as pd
import numpy as np
import re
from typing import Dict, List

## 2) Load ORS sheet as a raw grid


In [2]:
# ---- Optional: Colab Drive mount (safe to ignore locally) ----
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")
except Exception:
    pass

# ---- Inputs (edit as needed) ----
xlsx_path = "/content/drive/MyDrive/Next Round/ORS (2015_2017_2019) copied 2025-7-5.xlsx"
sheet_name = "HIV2015"

# Read the sheet as a raw Excel grid (no header), because ORS templates use multi-row headers
sheet_df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None)
print("Sheet shape:", sheet_df.shape)


Mounted at /content/drive
Sheet shape: (238, 84)


## 3) Helpers (Excel-style addressing + safe parsing)

The ORS sheets behave like an Excel grid:
- We use **Excel column letters** (e.g., `AP`, `AS`) and convert them to 0-based indices.
- We parse cells defensively because ORS sheets mix numbers, strings, blanks, and percentages.


In [3]:
def excel_col_to_idx(col: str) -> int:
    """Excel column letters -> 0-based index (A=0, B=1, ..., AP=41)."""
    col = col.strip().upper()
    n = 0
    for ch in col:
        if not ("A" <= ch <= "Z"):
            raise ValueError(f"Invalid Excel column: {col}")
        n = n * 26 + (ord(ch) - ord("A") + 1)
    return n - 1

def to_float(x):
    """Safe float parsing for ORS cells."""
    if x is None:
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).strip()
    if s == "" or s.lower() in {"nan", "none"}:
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

def as_fraction(x):
    """Normalize % values to fractions (72 -> 0.72); keep 0.xx as-is."""
    x = to_float(x)
    if np.isnan(x):
        return np.nan
    return x / 100.0 if x > 1.0 else x

def contains_drug(regimen, drug_token: str) -> bool:
    """Token-safe drug match inside regimen string."""
    if regimen is None or (isinstance(regimen, float) and np.isnan(regimen)):
        return False
    reg = str(regimen)
    pattern = r'(^|[\s\+/,;])' + re.escape(drug_token) + r'($|[\s\+/,;])'
    return re.search(pattern, reg) is not None

def regimen_size(regimen):
    """Fallback denominator when AX is missing: count '+' separated drugs."""
    if regimen is None or (isinstance(regimen, float) and np.isnan(regimen)):
        return np.nan
    parts = [p.strip() for p in str(regimen).split("+") if p.strip()]
    return float(len(parts)) if parts else np.nan

def find_row_contains_text(df, col_idx: int, text: str):
    """First row where df[r, col] contains text (case-insensitive)."""
    for r in range(len(df)):
        v = df.iat[r, col_idx]
        if isinstance(v, str) and text.lower() in v.lower():
            return r
    return None

def find_row_with_exact_value(df, col_idx: int, target: str):
    """First row where df[r, col] == target (string-exact after strip)."""
    s = df.iloc[:, col_idx].astype(str).str.strip()
    hits = np.where(s.values == target)[0]
    return int(hits[0]) if len(hits) else None

## 4) Entity map (anchored constants + regimen table columns)

This section defines:
- Fixed **column anchors** (e.g., regimen columns, constants columns)
- Fixed **row ranges** for regimen blocks (first-line / second-line)

The point is to keep the “map” readable and easy to audit (similar to the HIV 2013 entity-map style).


In [4]:
A = excel_col_to_idx("A")

afghan_row0 = find_row_with_exact_value(sheet_df, A, "Afghanistan")
if afghan_row0 is None:
    raise RuntimeError("Could not find Afghanistan in column A. Check file/sheet.")

country_rows = []
r = afghan_row0
while r < len(sheet_df):
    name = str(sheet_df.iat[r, A]).strip()
    if name == "" or name.lower() == "nan":
        break
    country_rows.append(r)
    r += 1

countries = [str(sheet_df.iat[r, A]).strip() for r in country_rows]
print("Countries:", len(countries), "| Example:", countries[:5])

Countries: 217 | Example: ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra']


In [5]:
AS = excel_col_to_idx("AS")
AT = excel_col_to_idx("AT")
AU = excel_col_to_idx("AU")
AV = excel_col_to_idx("AV")
AW = excel_col_to_idx("AW")
AX = excel_col_to_idx("AX")

first_hdr  = find_row_contains_text(sheet_df, AS, "first line")
second_hdr = find_row_contains_text(sheet_df, AS, "second line")

if first_hdr is None or second_hdr is None:
    raise RuntimeError("Could not find first/second-line headers in column AS.")

def collect_regimen_rows(start_row: int, end_row: int) -> List[int]:
    """Regimen rows are those with a regimen label + numeric AT or AV."""
    rows = []
    for rr in range(start_row, end_row):
        reg = sheet_df.iat[rr, AS]
        if reg is None or (isinstance(reg, float) and np.isnan(reg)):
            continue
        if (not np.isnan(to_float(sheet_df.iat[rr, AT]))) or (not np.isnan(to_float(sheet_df.iat[rr, AV]))):
            rows.append(rr)
    return rows

first_line_rows = collect_regimen_rows(first_hdr + 1, second_hdr)

end = second_hdr + 1
while end < len(sheet_df):
    v = sheet_df.iat[end, AS]
    if v is None or (isinstance(v, float) and np.isnan(v)):
        break
    end += 1

second_line_rows = collect_regimen_rows(second_hdr + 1, end)

print("First-line count:", len(first_line_rows), "| Example:", sheet_df.iat[first_line_rows[0], AS])
print("Second-line count:", len(second_line_rows), "| Example:", sheet_df.iat[second_line_rows[0], AS])

First-line count: 14 | Example: TDF + 3TC + EFV
Second-line count: 17 | Example: TDF + 3TC + LPV/r


In [6]:
AP = excel_col_to_idx("AP")

def cell(excel_row_1based: int, col_idx: int):
    return sheet_df.iat[excel_row_1based - 1, col_idx]

AP5  = as_fraction(cell(5,  AP))
AP6  = as_fraction(cell(6,  AP))
AP10 = as_fraction(cell(10, AP))
AP11 = as_fraction(cell(11, AP))

print("AP5, AP6, AP10, AP11 =", AP5, AP6, AP10, AP11)

AP5, AP6, AP10, AP11 = 0.90835 0.09165 0.89325 0.10675


## 5) Build a clean header row + identify drug columns

ORS uses multi-row headers. We build a **canonical header row** and then scan for columns that look like:
- `Impact of <drug>`

Those columns define the “universe of drugs” we compute impact scores for.


In [7]:
header_row = sheet_df.iloc[2].tolist()   # Excel row 3
header_map = {str(v).strip(): i for i, v in enumerate(header_row)}

drug_names = ["3TC","ABC","AZT","ddl","d4T","EFV","FTC","LPV/r","NVP","TDF","ATV/r"]
drug_cols: Dict[str, int] = {d: header_map[d] for d in drug_names if d in header_map}

print("Drug columns found:", drug_cols)

Drug columns found: {'3TC': 24, 'ABC': 25, 'AZT': 26, 'ddl': 27, 'd4T': 28, 'EFV': 29, 'FTC': 30, 'LPV/r': 31, 'NVP': 32, 'TDF': 33, 'ATV/r': 34}


## 6) Country-row inputs (DALYs, coverage, retention)

Each country row provides the per-country inputs required by the HIV impact model:
- Adult DALYs, Child DALYs
- Adult coverage, Child coverage
- Retention rate

These are read from the sheet once the header row and row indices are fixed.


In [8]:
G    = excel_col_to_idx("G")    # Adult DALYs
H    = excel_col_to_idx("H")    # Children DALYs
Icol = excel_col_to_idx("I")    # Retention (%)
Q    = excel_col_to_idx("Q")    # Adult coverage
T    = excel_col_to_idx("T")    # Child coverage

def impact_term(daly, AP_const, prop, coverage, efficacy, denom):
    """
    HIV2015 ORS formula:
      x = AP_const * prop * efficacy
      impact = daly * (coverage*x)/(1-coverage*x) / denom
    """
    daly     = to_float(daly)
    AP_const = as_fraction(AP_const)
    prop     = as_fraction(prop)
    coverage = as_fraction(coverage)
    efficacy = as_fraction(efficacy)
    denom    = to_float(denom)

    if any(np.isnan(z) for z in [daly, AP_const, prop, coverage, efficacy, denom]) or denom == 0:
        return 0.0

    x = AP_const * prop * efficacy
    prod = coverage * x
    d = 1.0 - prod
    if d == 0:
        return 0.0

    return (daly * prod / d) / denom

## 7) Impact Score formula (per country × per drug)

Core idea:
1. For a given **drug**, scan regimen strings in the regimen tables.
2. When the regimen contains the drug, add its contribution:
   - adult first-line + adult second-line
   - child first-line + child second-line
3. Apply **retention-rate normalization** at the end.

This mirrors the HIV model described in the supporting info:
Impact is computed via a coverage×efficacy style term and then normalized using retention rate.


In [9]:
def compute_country_drug(country_row_idx: int, drug_token: str) -> float:
    adult_dalys = sheet_df.iat[country_row_idx, G]
    child_dalys = sheet_df.iat[country_row_idx, H]
    adult_cov   = sheet_df.iat[country_row_idx, Q]
    child_cov   = sheet_df.iat[country_row_idx, T]
    retention   = to_float(sheet_df.iat[country_row_idx, Icol])

    total = 0.0

    for rr in first_line_rows:
        reg = sheet_df.iat[rr, AS]
        if contains_drug(reg, drug_token):
            denom = to_float(sheet_df.iat[rr, AX])
            if np.isnan(denom) or denom == 0:
                denom = regimen_size(reg)
            total += impact_term(adult_dalys, AP5,  sheet_df.iat[rr, AT], adult_cov, sheet_df.iat[rr, AU], denom)
            total += impact_term(child_dalys, AP10, sheet_df.iat[rr, AV], child_cov, sheet_df.iat[rr, AW], denom)

    for rr in second_line_rows:
        reg = sheet_df.iat[rr, AS]
        if contains_drug(reg, drug_token):
            denom = to_float(sheet_df.iat[rr, AX])
            if np.isnan(denom) or denom == 0:
                denom = regimen_size(reg)
            total += impact_term(adult_dalys, AP6,  sheet_df.iat[rr, AT], adult_cov, sheet_df.iat[rr, AU], denom)
            total += impact_term(child_dalys, AP11, sheet_df.iat[rr, AV], child_cov, sheet_df.iat[rr, AW], denom)

    # retention normalization
    if np.isnan(retention):
        return 0.0
    norm = 100.0 / (100.0 - retention) if (100.0 - retention) != 0 else np.nan
    return total / norm if not np.isnan(norm) and norm != 0 else 0.0

## 8) Validation against Excel

For each drug:
- compute all country values
- read the corresponding Excel column values
- print max/mean absolute differences

A successful run should show very small diffs (floating noise only).


In [10]:
results_wide = pd.DataFrame({"Country": countries})

for drug, col_idx in drug_cols.items():
    computed_vals = [compute_country_drug(r, drug) for r in country_rows]
    results_wide[f"Computed_{drug}"] = computed_vals

    excel_vals = [to_float(sheet_df.iat[r, col_idx]) for r in country_rows]
    results_wide[f"Excel_{drug}"] = excel_vals

    diff = np.array(computed_vals, dtype=float) - np.array(excel_vals, dtype=float)
    print(f"{drug}: max_abs_diff={np.nanmax(np.abs(diff)):.12f}, mean_abs_diff={np.nanmean(np.abs(diff)):.12f}")

results_wide.head(10)

3TC: max_abs_diff=0.054960460687, mean_abs_diff=0.000253532978
ABC: max_abs_diff=0.000219287938, mean_abs_diff=0.000001053634
AZT: max_abs_diff=0.020440843554, mean_abs_diff=0.000094365736
ddl: max_abs_diff=0.000000048454, mean_abs_diff=0.000000000767
d4T: max_abs_diff=0.000000469909, mean_abs_diff=0.000000003243
EFV: max_abs_diff=0.042521293850, mean_abs_diff=0.000196124546
FTC: max_abs_diff=0.015477831448, mean_abs_diff=0.000071414092
LPV/r: max_abs_diff=0.004610894755, mean_abs_diff=0.000021296756
NVP: max_abs_diff=0.022443534331, mean_abs_diff=0.000103562296
TDF: max_abs_diff=0.051344735204, mean_abs_diff=0.000236826016
ATV/r: max_abs_diff=0.000862569199, mean_abs_diff=0.000003979231


,Country,Computed_3TC,Excel_3TC,Computed_ABC,Excel_ABC,Computed_AZT,Excel_AZT,Computed_ddl,Excel_ddl,Computed_d4T,...,Computed_FTC,Excel_FTC,Computed_LPV/r,Excel_LPV/r,Computed_NVP,Excel_NVP,Computed_TDF,Excel_TDF,Computed_ATV/r,Excel_ATV/r
0,Afghanistan,35.667802,35.667802,3.483114,3.483114,15.831563,15.831563,0.102642,0.102642,0.523836,...,7.057698,7.057698,4.612259,4.612259,16.425901,16.425901,23.749159,23.749159,0.412628,0.412628
1,Albania,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Algeria,186.166832,186.166832,3.714239,3.714239,70.801802,70.801802,0.088267,0.088267,0.455925,...,49.132222,49.132222,15.754865,15.754865,77.092066,77.092066,164.992819,164.992819,2.513475,2.513475
3,American Samoa,0.054960,0.000000,0.000219,0.000000,0.020441,0.000000,0.000000,0.000000,0.000000,...,0.015478,0.000000,0.004611,0.000000,0.022444,0.000000,0.051345,0.000000,0.000863,0.000000
4,Andorra,0.176927,0.176927,0.000706,0.000706,0.065802,0.065802,0.000000,0.000000,0.000000,...,0.049826,0.049826,0.014843,0.014843,0.072249,0.072249,0.165287,0.165287,0.002777,0.002777
5,Angola,1085.331064,1085.331064,84.823245,84.823245,466.427030,466.427030,2.455045,2.455045,12.552467,...,232.399539,232.399539,128.131075,128.131075,489.156981,489.156981,780.409167,780.409167,13.092495,13.092495
6,Anguilla,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Antigua and Barbuda,0.826518,0.826518,0.003298,0.003298,0.307398,0.307398,0.000000,0.000000,0.000000,...,0.232762,0.232762,0.069341,0.069341,0.337515,0.337515,0.772143,0.772143,0.012972,0.012972
8,Argentina,3783.175240,3783.175240,122.256802,122.256802,1502.665106,1502.665107,3.136901,3.136901,16.203024,...,953.980101,953.980102,350.738827,350.738827,1618.611918,1618.611918,3192.017671,3192.017671,50.927850,50.927850
9,Armenia,5.906418,5.906418,0.023988,0.023988,2.206146,2.206146,0.000000,0.000000,0.000000,...,1.663950,1.663950,0.503058,0.503058,2.419804,2.419804,5.511532,5.511532,0.094334,0.094334


## 9) Export `impact_score.csv`

Final deliverable for the task:
- `Country`
- `Impact Score (<drug>)` columns for all drugs in HIV 2015 ORS


In [12]:
impact_cols = ["Country"] + [f"Computed_{d}" for d in drug_cols.keys()]
impact_score = results_wide[impact_cols].copy()
impact_score.columns = ["Country"] + [f"Impact Score ({d})" for d in drug_cols.keys()]

impact_score.to_csv("impact_score.csv", index=False)
print("Saved:", "impact_score.csv", "| shape:", impact_score.shape)

impact_score.head(10)

Saved: impact_score.csv | shape: (217, 12)


,Country,Impact Score (3TC),Impact Score (ABC),Impact Score (AZT),Impact Score (ddl),Impact Score (d4T),Impact Score (EFV),Impact Score (FTC),Impact Score (LPV/r),Impact Score (NVP),Impact Score (TDF),Impact Score (ATV/r)
0,Afghanistan,35.667802,3.483114,15.831563,0.102642,0.523836,21.363380,7.057698,4.612259,16.425901,23.749159,0.412628
1,Albania,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Algeria,186.166832,3.714239,70.801802,0.088267,0.455925,140.014900,49.132222,15.754865,77.092066,164.992819,2.513475
3,American Samoa,0.054960,0.000219,0.020441,0.000000,0.000000,0.042521,0.015478,0.004611,0.022444,0.051345,0.000863
4,Andorra,0.176927,0.000706,0.065802,0.000000,0.000000,0.136883,0.049826,0.014843,0.072249,0.165287,0.002777
5,Angola,1085.331064,84.823245,466.427030,2.455045,12.552467,689.470875,232.399539,128.131075,489.156981,780.409167,13.092495
6,Anguilla,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Antigua and Barbuda,0.826518,0.003298,0.307398,0.000000,0.000000,0.639453,0.232762,0.069341,0.337515,0.772143,0.012972
8,Argentina,3783.175240,122.256802,1502.665106,3.136901,16.203024,2719.586642,953.980101,350.738827,1618.611918,3192.017671,50.927850
9,Armenia,5.906418,0.023988,2.206146,0.000000,0.000000,4.553171,1.663950,0.503058,2.419804,5.511532,0.094334


---
### Notes for reviewers

- This notebook intentionally reads the ORS sheet as a **raw grid** (`header=None`) to mirror Excel.
- The entity map is kept **explicit and auditable** (anchors + row ranges), so a reviewer can trace each term back to the ORS.
- The exported `impact_score.csv` is the submission artifact requested by the task.
